# 02_demo_and_streamlit.ipynb

This notebook loads the trained CO₂ prediction model,
tests predictions, simulates a CSV-based carbon credit marketplace,
and generates a Streamlit app (`app/streamlit_app.py`).



In [1]:
import joblib
import numpy as np
import pandas as pd
import os

# Load trained model
model_path = "../model/model.joblib"
model = joblib.load(model_path)

print("✅ Model loaded from", model_path)

# Quick test prediction
sample = np.array([[3600, 5000, 200, 1]])  # energy_kwh, transport_km, waste_kg, flights
pred = model.predict(sample)[0]
print(f"Predicted CO₂ for sample household: {pred:.2f} tons")


✅ Model loaded from ../model/model.joblib
Predicted CO₂ for sample household: 2.22 tons


C:\Users\Abhay\carboncredits-ai\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [2]:
# Marketplace CSVs
market_csv = "../data/marketplace.csv"
trans_csv = "../data/transactions.csv"

# Create sample listings
market = pd.DataFrame([
    {"id": "L1", "seller": "eco_org", "credits_tons": 10.0, "price_per_ton": 300.0},
    {"id": "L2", "seller": "green_farm", "credits_tons": 5.0, "price_per_ton": 280.0},
])
market.to_csv(market_csv, index=False)

# Empty transactions file
pd.DataFrame(columns=["tx_id","listing_id","seller","buyer","credits_tons","price_per_ton"]).to_csv(trans_csv, index=False)

print("✅ Marketplace and transactions CSVs ready")
pd.read_csv(market_csv)


✅ Marketplace and transactions CSVs ready


,id,seller,credits_tons,price_per_ton
0,L1,eco_org,10.0,300.0
1,L2,green_farm,5.0,280.0


In [3]:
# Load marketplace
market = pd.read_csv(market_csv)
print("Available listings:\n", market)

# Choose first listing
listing = market.iloc[0]

# Simulate a buy
tx = {
    "tx_id": pd.Timestamp.now().strftime("%Y%m%d%H%M%S"),
    "listing_id": listing["id"],
    "seller": listing["seller"],
    "buyer": "notebook_buyer",
    "credits_tons": listing["credits_tons"],
    "price_per_ton": listing["price_per_ton"],
}

# Append transaction
pd.DataFrame([tx]).to_csv(trans_csv, mode="a", header=False, index=False)

# Remove listing from marketplace
market = market[market["id"] != listing["id"]]
market.to_csv(market_csv, index=False)

print("✅ Purchase simulated and recorded")
pd.read_csv(trans_csv).tail()


Available listings:
    id      seller  credits_tons  price_per_ton
0  L1     eco_org          10.0          300.0
1  L2  green_farm           5.0          280.0
✅ Purchase simulated and recorded


,tx_id,listing_id,seller,buyer,credits_tons,price_per_ton
0,20250913131513,L1,eco_org,notebook_buyer,10.0,300.0


In [4]:
st_code = r'''
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os

st.set_page_config(page_title="CarbonCredits.ai", layout="wide")
st.title("🌍 CarbonCredits.ai — CO₂ Footprint & Marketplace Demo")

# Load model
MODEL_PATH = "../model/model.joblib" if os.path.exists("../model/model.joblib") else "model/model.joblib"
if not os.path.exists(MODEL_PATH):
    st.error("Model not found. Run the training notebook first.")
    st.stop()
model = joblib.load(MODEL_PATH)

# --- Section 1: Prediction ---
st.header("1. Estimate your annual CO₂ emissions")
col1, col2 = st.columns(2)
with col1:
    energy_kwh = st.number_input("Annual electricity (kWh)", min_value=0.0, value=3600.0, step=10.0)
    transport_km = st.number_input("Annual transport (km)", min_value=0.0, value=5000.0, step=10.0)
with col2:
    waste_kg = st.number_input("Annual waste (kg)", min_value=0.0, value=200.0, step=1.0)
    flights = st.number_input("Flights per year", min_value=0, value=1, step=1)

features = np.array([[energy_kwh, transport_km, waste_kg, flights]])
prediction = model.predict(features)[0]
prediction = max(prediction, 0.0)

st.metric("Estimated CO₂ emissions", f"{prediction:.2f} tons")
st.info(f"Recommended credits: {prediction:.2f}")

# --- Section 2: Marketplace ---
st.markdown("---")
st.header("2. Carbon Credit Marketplace (Demo)")

MARKET_CSV = "../data/marketplace.csv" if os.path.exists("../data/marketplace.csv") else "data/marketplace.csv"
TRANS_CSV = "../data/transactions.csv" if os.path.exists("../data/transactions.csv") else "data/transactions.csv"
os.makedirs(os.path.dirname(MARKET_CSV), exist_ok=True)

# Add listing
with st.expander("List credits for sale"):
    with st.form("list_form"):
        seller = st.text_input("Your name", value="seller1")
        credits_tons = st.number_input("Credits (tons)", min_value=0.1, value=1.0, step=0.1)
        price_per_ton = st.number_input("Price per ton", min_value=0.1, value=300.0, step=1.0)
        submit = st.form_submit_button("List for sale")
    if submit:
        df_new = pd.DataFrame([{
            "id": pd.Timestamp.now().strftime("%Y%m%d%H%M%S"),
            "seller": seller,
            "credits_tons": credits_tons,
            "price_per_ton": price_per_ton
        }])
        if os.path.exists(MARKET_CSV):
            df_new.to_csv(MARKET_CSV, mode="a", header=False, index=False)
        else:
            df_new.to_csv(MARKET_CSV, index=False)
        st.success("✅ Listing added!")

# Show listings
if os.path.exists(MARKET_CSV):
    market_df = pd.read_csv(MARKET_CSV)
    st.dataframe(market_df)

# Buy listing
if os.path.exists(MARKET_CSV) and not pd.read_csv(MARKET_CSV).empty:
    market_df = pd.read_csv(MARKET_CSV)
    options = market_df["id"].astype(str).tolist()
    pick = st.selectbox("Select listing ID to buy", options=options)
    buyer = st.text_input("Buyer name", value="buyer1")
    if st.button("Buy"):
        row = market_df[market_df["id"].astype(str) == pick].iloc[0]
        trans = pd.DataFrame([{
            "tx_id": pd.Timestamp.now().strftime("%Y%m%d%H%M%S"),
            "listing_id": row["id"],
            "seller": row["seller"],
            "buyer": buyer,
            "credits_tons": row["credits_tons"],
            "price_per_ton": row["price_per_ton"]
        }])
        if os.path.exists(TRANS_CSV):
            trans.to_csv(TRANS_CSV, mode="a", header=False, index=False)
        else:
            trans.to_csv(TRANS_CSV, index=False)
        # remove from marketplace
        market_df = market_df[market_df["id"].astype(str) != pick]
        market_df.to_csv(MARKET_CSV, index=False)
        st.success("✅ Purchase completed!")
'''

os.makedirs("../app", exist_ok=True)
with open("../app/streamlit_app.py", "w", encoding="utf-8") as f:
    f.write(st_code)

print("✅ Streamlit app written to ../app/streamlit_app.py")


✅ Streamlit app written to ../app/streamlit_app.py


# ▶️ How to run the Streamlit app

In terminal (PowerShell or JupyterLab terminal):

```bash
cd carboncredits-ai
streamlit run app/streamlit_app.py
